In [27]:
import os
import json
import numpy as np
import pandas as pd

from datetime import datetime
from sqlalchemy import create_engine

np.set_printoptions(linewidth=100000)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 500)

user = 'admin'
passw = 'Y3wR3view9!'
host = 'yew-review-test.c2usv7nad3n6.us-west-2.rds.amazonaws.com'
port = 3306
database = 'yewreview'
table = 'articles'

engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(user, passw, host, port, database), encoding='utf8', convert_unicode=True)


query = """
 SELECT uri, scrape_date
   FROM articles
  WHERE publisher = 'stabmag'
"""

In [57]:
df = pd.read_sql(query, engine)

In [58]:
df[df.uri.str.endswith('/')].shape[0]

4148

In [59]:
df[~pd.isnull(df.scrape_date)].shape[0]

4148

In [60]:
counts = df.fillna(0).groupby('uri').count().reset_index()
counts

,uri,scrape_date
0,https://stabmag.com/news/1-5-million-the-wsl-and-a-new-program-to-save-our-oceans/,1
1,https://stabmag.com/news/10-men-stab-wants-to-hear-in-the-commentary-booth/,1
2,https://stabmag.com/news/10-of-the-best-waves-ever-ridden-at-cloudbreak/,1
3,https://stabmag.com/news/10-of-the-worlds-sharkiest-spots-that-you-cant-help-but-surf/,1
4,https://stabmag.com/news/10-photos-of-dane-reynolds-testing-the-worlds-best-surfboards/,1
5,https://stabmag.com/news/10-photos-of-jordy-smith-testing-the-worlds-best-surfboards/,1
6,https://stabmag.com/news/10-questions-you-should-never-ask-an-indo-boat-captain/,1
7,https://stabmag.com/news/10-reasons-why-2014-will-be-better-than-2013/,1
8,https://stabmag.com/news/10-reasons-why-you-need-to-see-nic-nix-nooley/,1
9,https://stabmag.com/news/10-surfers-you-want-at-your-party/,1


In [61]:
counts[counts.scrape_date > 1]

,uri,scrape_date


In [62]:
uri = "https://stabmag.com/news/massive-layoffs-come-to-hurley/"
df[df.uri == uri]

,uri,scrape_date
1070,https://stabmag.com/news/massive-layoffs-come-to-hurley/,2020-01-17 07:58:30


In [63]:
for tup in counts[counts.scrape_date > 1].itertuples():
    print(tup.uri)
    print(tup.scrape_date)

In [51]:
drops = df.merge(counts[counts.scrape_date > 1], how='inner', on='uri')
drops = drops.groupby('uri').scrape_date_x.min().reset_index()
drops

,uri,scrape_date_x
0,https://stabmag.com/news/channel-islands-is-having-a-party-tonight-to-celebrate-its-double-board-launch/,2020-01-15 21:18:42
1,https://stabmag.com/news/interview-how-to-dominate-a-cyclone-swell-with-jack-freestone/,2020-01-15 22:23:29
2,https://stabmag.com/news/john-john-and-pyzel-are-raffling-off-a-board-for-aussie-fire-relief/,2020-01-15 21:18:42
3,https://stabmag.com/news/massive-layoffs-come-to-hurley/,2020-01-17 07:58:11
4,https://stabmag.com/news/shuv-it-pioneer-zach-rhinehart-is-battling-cancer/,2020-01-16 07:02:08


In [55]:
DELETE_QUERY = """
DELETE FROM articles WHERE uri='{}' AND scrape_date='{}'
"""

In [56]:
with engine.connect() as con:
    for drop in drops.itertuples():
        print('Deleting: {} {}'.format(drop.uri, drop.scrape_date_x))
        con.execute(DELETE_QUERY.format(drop.uri, drop.scrape_date_x))

Deleting: https://stabmag.com/news/channel-islands-is-having-a-party-tonight-to-celebrate-its-double-board-launch/ 2020-01-15 21:18:42
Deleting: https://stabmag.com/news/interview-how-to-dominate-a-cyclone-swell-with-jack-freestone/ 2020-01-15 22:23:29
Deleting: https://stabmag.com/news/john-john-and-pyzel-are-raffling-off-a-board-for-aussie-fire-relief/ 2020-01-15 21:18:42
Deleting: https://stabmag.com/news/massive-layoffs-come-to-hurley/ 2020-01-17 07:58:11
Deleting: https://stabmag.com/news/shuv-it-pioneer-zach-rhinehart-is-battling-cancer/ 2020-01-16 07:02:08
